In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import r2_score
import pandas as pd
import numpy as np

In [5]:
# Load dataset
df = pd.read_csv('/content/laptop_price - dataset.csv')

# Split 
X = df.drop('Price (Euro)', axis=1)
y = df['Price (Euro)']

# List of categorical columns
categorical_cols = ['Company', 'Product', 'TypeName', 'ScreenResolution',
                    'CPU_Company', 'CPU_Type', 'Memory', 'GPU_Company',
                    'GPU_Type', 'OpSys']

# List of numerical columns
numerical_cols = ['Inches', 'CPU_Frequency (GHz)', 'RAM (GB)', 'Weight (kg)']

# Preprocessor to handle both categorical and numerical columns
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_cols),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols)
    ])

# Preprocess the data 
X_preprocessed = preprocessor.fit_transform(X)

# Convert to tensors
X_tensor = torch.tensor(X_preprocessed.toarray(), dtype=torch.float32)
y_tensor = torch.tensor(y.values, dtype=torch.float32).view(-1, 1)


In [6]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(X_tensor, y_tensor, test_size=0.2, random_state=42)

# Create training and testing sets
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

# Create DataLoaders for batching
train_loader = DataLoader(dataset=train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=32, shuffle=False)


In [7]:
# Define the Model
class LinearRegressionModel(nn.Module):
    def __init__(self, input_dim):
        super(LinearRegressionModel, self).__init__()
        self.linear = nn.Linear(input_dim, 100)
        self.relu = nn.ReLU()
        self.linear2 = nn.Linear(100, 1)

    def forward(self, x):
        out = self.linear(x)
        out = self.relu(out)
        out = self.linear2(out)
        return out


In [8]:

# Initialize the model
input_dim = X_train.shape[1]
model = LinearRegressionModel(input_dim)

# Define loss function and optimizer
criterion = nn.MSELoss()  
optimizer = optim.Adam(model.parameters(), lr=0.01)  


In [9]:
# Training the Model
num_epochs = 1000
for epoch in range(num_epochs):
    model.train()  
    for batch_X, batch_y in train_loader:  
        optimizer.zero_grad()

        # Forward pass
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

    if (epoch + 1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

Epoch [100/1000], Loss: 11548.4199
Epoch [200/1000], Loss: 4136.7920
Epoch [300/1000], Loss: 3429.7136
Epoch [400/1000], Loss: 3757.5042
Epoch [500/1000], Loss: 9679.5225
Epoch [600/1000], Loss: 2350.7661
Epoch [700/1000], Loss: 10573.1309
Epoch [800/1000], Loss: 631.6229
Epoch [900/1000], Loss: 8194.0322
Epoch [1000/1000], Loss: 2591.2163


In [10]:
# Evaluation the Model using R²
model.eval()  
with torch.no_grad():  
    y_pred_list = []
    y_true_list = []
    for batch_X, batch_y in test_loader:
        # Get model predictions
        y_pred = model(batch_X)
        y_pred_list.append(y_pred)
        y_true_list.append(batch_y)

    # Concatenate all batches into a single tensor
    y_pred_tensor = torch.cat(y_pred_list, dim=0)
    y_true_tensor = torch.cat(y_true_list, dim=0)

    # Calculate R² score (accuracy for regression tasks)
    y_pred_numpy = y_pred_tensor.numpy()
    y_true_numpy = y_true_tensor.numpy()

    r2 = r2_score(y_true_numpy, y_pred_numpy)
    print(f'R² Score (Accuracy): {r2:.4f}')

R² Score (Accuracy): 0.8345


In [14]:
def custom_test(model, custom_input, preprocessor):

    model.eval()  

    custom_input_df = pd.DataFrame([custom_input])  # Convert the custom input to a DataFrame

    custom_input_preprocessed = preprocessor.transform(custom_input_df)  # Assuming custom_input is a single data point

    custom_input_tensor = torch.tensor(custom_input_preprocessed.toarray(), dtype=torch.float32)

    with torch.no_grad():  
        prediction = model(custom_input_tensor)

    print(f'Predicted Price (Euro): {prediction.item():.2f}')

# Example custom test
custom_input = {
    'Company': 'Dell',
    'Product': 'XPS 13',
    'TypeName': 'Ultrabook',
    'Inches': 13.3,
    'ScreenResolution': '1920x1080',
    'CPU_Company': 'Intel',
    'CPU_Type': 'Core i7',
    'CPU_Frequency (GHz)': 2.8,
    'RAM (GB)': 16,
    'Memory': '512GB SSD',
    'GPU_Company': 'Intel',
    'GPU_Type': 'Intel Iris',
    'OpSys': 'Windows 10',
    'Weight (kg)': 1.2
}

custom_test(model, custom_input, preprocessor)

Predicted Price (Euro): 2222.50
